# FT用データ生成スクリプト

In [1]:
# !conda install -y -c conda-forge kalpy \
# kaldi \
# pynini

# # パッケージインストール
# !pip install -r requirements.sbv.txt

In [2]:
# !pip list

In [3]:
# # mfa
# # 日本語辞書のダウンロード
# !mfa model download dictionary japanese_mfa

# # 日本語音響モデルのダウンロード
# !mfa model download acoustic japanese_mfa

## テキスト対話データ生成

In [4]:
import os
from typing import Literal
import ast

from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PDFMinerLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI


# .envファイル読み込み
load_dotenv()

/nfs1/s1f102201582/anaconda3/envs/sbv-tts/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [5]:
#config
from os.path import join, expanduser

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
BASE_URL = "https://api.openai.iniad.org/api/v1"
MODEL='gemini-2.5-flash'
TEMPERATURE = 1.0
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

# 生成する音声のサンプリングレート
setting_sr = 16000

#対話音声データの個数を指定
gen_dial_num = 1

# すでに作成した対話データを削除するかどうか
IS_REMOVE_EXIST_FILE = True

# ftに使うjsonとaudioの出力フォルダパス
home_dir = expanduser("~")
json_dir_path = join(home_dir, "Github/jmoshi-ft/gen_dialogue/data/sbv/transcription")
audio_dir_path = join(home_dir, "Github/jmoshi-ft/gen_dialogue/data/sbv/audio")

# mfa関連のパス
model_dir = join(home_dir, "Documents/MFA/pretrained_models/acoustic/japanese_mfa.zip")
mfa_input_dir = join(home_dir, "Github/jmoshi-ft/gen_dialogue/data/sbv/mfa_input")
mfa_output_dir = join(home_dir, "Github/jmoshi-ft/gen_dialogue/data/sbv/mfa_output")

In [6]:
base_paths = [
    json_dir_path,
    audio_dir_path,
    mfa_input_dir,
    mfa_output_dir,
]

for p in base_paths:
    if not os.path.isdir(p):
        os.makedirs(p)

In [7]:
# model定義
model = ChatGoogleGenerativeAI(
                 model=MODEL,
                 temperature=TEMPERATURE)

# 埋め込みモデル定義
embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=BASE_URL,
    model="text-embedding-3-large"
)

# データベース定義
vector_store = Chroma(
    collection_name="collection",
    embedding_function=embeddings,
    # persist_directory = "/path/to/db_file" # if necessary
)

In [8]:
loader = DirectoryLoader(
    "../../mental_docs/",
    glob="*.pdf",
    show_progress=True,
    loader_cls=PDFMinerLoader,
)
docs = loader.load()
print(f"Loaded {len(docs)} documents")

  0%|                                                                | 0/3 [00:00<?, ?it/s]Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color 

Loaded 3 documents


In [9]:
# Debug
# for doc in docs:
#     print("-------------------------------------------------")
#     print(doc.metadata)
#     print(len(doc.page_content))
#     print(doc.page_content[:100])

In [10]:
#読み込んだ文章データをオーバーラップ200文字で1000文字づつ分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True, # 分割前の文章のインデックスを追跡
)
splits = text_splitter.split_documents(docs)

# データベースにデータを追加
document_ids = vector_store.add_documents(documents=splits)

In [11]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query, k=2)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message

In [12]:
from typing import Literal

from pydantic import BaseModel, Field


class Dialogue(BaseModel):
    """対話データを構成する対話クラス"""
    speaker: Literal["A", "B"] = Field(..., description="話者。Aはカウンセラー、Bはクライエントを表す。")
    text: str = Field(..., description="話者が話した内容。")

class Dialogues(BaseModel):
    """カウンセリングを目的としたカウンセリング対話データ"""
    dialogues: list[Dialogue] = Field(..., description="対話データを構成する対話クラスのリスト。")

In [13]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model, 
    tools=[],
    middleware=[prompt_with_context],
    response_format=ToolStrategy(
        Dialogues,
        handle_errors="フォーマットに合うように、もう一度対話データを生成してください。"
    )
)

In [14]:
#promptを作成
import random


sessions = [
    "【段階：初期】信頼関係を築きつつ、悩みの背景を深掘りするシーン",
    "【段階：中期】クライエントの「すべき思考」に焦点を当て、認知の歪みを扱うシーン",
    "【段階：終結期】これまでのセッションを振り返り、終結に向けて準備するシーン",
]

def gen_prompt_txt():
    choiced = random.randint(0, 2)
    choiced_session = sessions[choiced]
    prompt_txt = f"""メンタルヘルスケアカウンセリングのセッションをシミュレーションしてください。
シミュレーションしたい「段階」と「テーマ」:
{choiced_session}

役割定義:
A (カウンセラー): メンタルヘルスケアの専門知識を持つ経験豊富なカウンセラー。傾聴と共感の姿勢を基本とし、クライエントの言葉を促すように、優しく、自然な話し言葉（「〜ですね」「〜でしたか」など）を使います。
B (クライエント): 仕事上の悩みだけでなく、日常生活全般に対して漠然とした不安や焦りを感じている人物。

対話の要件:
スタイル: 実際の会話の文字起こしのように、堅苦しくない自然な「話し言葉」を使用してください。
相槌 (あいづち): カウンセラー（A）は、クライエント（B）の話を促し、共感を示すため、「ええ」「はい」「そうなんですね」「なるほど」といった細かな相槌を頻繁に、適切なタイミングで挿入してください。
構成: 会話が途中で途切れるのではなく、初回のヒアリングとして「一区切り」がつき、自然に終了する流れにしてください（例：次回の約束、今回のまとめなど）。
分量: 会話の往復は合計12〜20ターン程度、全体の文字数が合計500〜800文字程度になるように構成してください。
"""
    return prompt_txt

In [15]:
# テキスト対話生成関数
def gen_txt_dialogue():
    prompt = gen_prompt_txt()
    resp = agent.invoke({"messages": [{"role": "user", "content": prompt}]})
    dialogues_list = resp["structured_response"].dialogues
    return dialogues_list

In [16]:
#DEBUG
# txt_dialogue = gen_txt_dialogue()
# print(txt_dialogue)
# lst_dialogue = txt_to_lst(txt_dialogue)
# print(lst_dialogue)

## テキスト対話データを音声対話データに変換 

In [17]:
from style_bert_vits2.nlp import bert_models
from style_bert_vits2.constants import Languages
from pathlib import Path
from huggingface_hub import hf_hub_download
from style_bert_vits2.tts_model import TTSModel

bert_models.load_model(Languages.JP, "ku-nlp/deberta-v2-large-japanese-char-wwm")
bert_models.load_tokenizer(Languages.JP, "ku-nlp/deberta-v2-large-japanese-char-wwm")
assets_root = Path("model_assets")

# # 子春音あみ
# model_file = "koharune-ami/koharune-ami.safetensors"
# config_file = "koharune-ami/config.json"
# style_file = "koharune-ami/style_vectors.npy"
# hf_repo = "litagin/sbv2_koharune_ami"

# # あみたろ
# model_file = "amitaro/amitaro.safetensors"
# config_file = "amitaro/config.json"
# style_file = "amitaro/style_vectors.npy"
# hf_repo = "litagin/sbv2_amitaro"


# デフォルトの女性2
model_file = "jvnv-F2-jp/jvnv-F2_e166_s20000.safetensors"
config_file = "jvnv-F2-jp/config.json"
style_file = "jvnv-F2-jp/style_vectors.npy"
hf_repo = "litagin/style_bert_vits2_jvnv"

for file in [model_file, config_file, style_file]:
    print(file)
    hf_hub_download(hf_repo, file, local_dir="model_assets")

A_model = TTSModel(
    model_path=assets_root / model_file,
    config_path=assets_root / config_file,
    style_vec_path=assets_root / style_file,
    device="cuda",
)

# デフォルトの男性2
model_file = "jvnv-M2-jp/jvnv-M2-jp_e159_s17000.safetensors"
config_file = "jvnv-M2-jp/config.json"
style_file = "jvnv-M2-jp/style_vectors.npy"

for file in [model_file, config_file, style_file]:
    print(file)
    hf_hub_download(hf_repo, file, local_dir="model_assets")

B_model = TTSModel(
    model_path=assets_root / model_file,
    config_path=assets_root / config_file,
    style_vec_path=assets_root / style_file,
    device="cuda",
)

10-26 00:34:14 |  INFO  | bert_models.py:92 | Loaded the Languages.JP BERT model from ku-nlp/deberta-v2-large-japanese-char-wwm
10-26 00:34:17 |  INFO  | bert_models.py:154 | Loaded the Languages.JP BERT tokenizer from ku-nlp/deberta-v2-large-japanese-char-wwm
jvnv-F2-jp/jvnv-F2_e166_s20000.safetensors
jvnv-F2-jp/config.json
jvnv-F2-jp/style_vectors.npy
jvnv-M2-jp/jvnv-M2-jp_e159_s17000.safetensors
jvnv-M2-jp/config.json
jvnv-M2-jp/style_vectors.npy


In [31]:
def build_audio_synth_prompt(text_dialogue_list):
    resp = ""
    resp_header =  """あなたがこれから音声合成するテキストは以下の対話内容のワンフレーズです。
この対話の文脈に合うように音声合成してください。

<対話内容の全文>"""
    resp += resp_header
    for text_dial in text_dialogue_list:
        resp += f"\n{text_dial.speaker}: {text_dial.text}"
    print(resp)
    return resp

In [19]:
from typing import Literal

def sbv_tts(text: str, speaker: Literal["A", "B"], assist_text=None):
    if speaker == "A":
        sr, audio = A_model.infer(
            text = text,
            style='Happy',
            style_weight=1,
            split_interval = 0.3,
            use_assist_text = True if assist_text is not None else None,
            assist_text = assist_text
        )
    else:
        sr, audio = B_model.infer(
            text = text,
            style='Sad',
            style_weight=1,
            split_interval = 0.3,
            use_assist_text = True if assist_text is not None else None,
            assist_text = assist_text
        )
    
    return sr, audio

In [28]:
import librosa
import numpy as np

def gen_audio_dialogue(text_dialogue_list, prompt):
    # 音声ファイルを順番に生成（ファイルは不要なのでwave配列で持つ）
    wav_data = []
    for dial in text_dialogue_list:
        speaker = dial.speaker
        sr, wav = sbv_tts(dial.text, speaker, prompt)

        # サンプリングレートを変換
        if sr != setting_sr:
            # 16ビット整数のデータを、-1.0から1.0の範囲に収まる浮動小数点数に正規化
            wav = wav.astype(np.float32) / 32768.0
            wav = librosa.resample(wav, orig_sr=sr, target_sr=setting_sr)

        # 0.3秒間の無音時間を追加
        duration_sec = 0.3
        num_silent_samples = int(setting_sr*duration_sec)
        silence = np.zeros(num_silent_samples, dtype=wav.dtype)
        wav_with_silence = np.concatenate((wav, silence))
        wav_data.append(wav_with_silence)
    
    # 最終的な音声長を決定
    max_len = sum([len(w) for w in wav_data])
    
    # ステレオ音声用（2チャンネル×最大長）の空配列をゼロ初期化で作成
    stereo = np.zeros((2, max_len), dtype=np.float32)
    
    pos = 0
    for i, wav in enumerate(wav_data):
        ch = i%2  # 0:左(A), 1:右(B)
        stereo[ch, pos:pos+len(wav)] += wav
        pos += len(wav)
    
    # 転置(-1,2)する
    stereo = stereo.T
    return stereo

## mfa(montreal force alignment)による音声アラインメント

In [21]:
import MeCab
import re

# 句読点のパターン
PUNCT_RE = re.compile(r'^[。、,.!?！？…]+$')

def tokenize_text(text, is_punct_isolated=False):
    tokens = []
    punct_array = []
    checked_punct_pos = 0
    try:
        # MeCabのタガーを初期化
        tagger = MeCab.Tagger()

        # MeCabは内部でShift-JISやEUC-JPを期待することがあるため、
        # UnicodeDecodeErrorを避けるために明示的にUTF-8でエンコード・デコードする
        # parseToNodeは、より詳細な情報をノードオブジェクトとして取得できるメソッド
        node = tagger.parseToNode(text)
        while node:
            if not node.surface:
                pass
                
            elif not is_punct_isolated and PUNCT_RE.match(node.surface) and tokens:
                punct_array.append([checked_punct_pos, node.surface])
                
                checked_punct_pos += len(node.surface)
                # 句読点なら直前のトークンに連結
                tokens[-1] += node.surface
            else:
                checked_punct_pos += len(node.surface)
                
                # 通常トークンはそのまま追加
                tokens.append(node.surface)
            node = node.next
    except RuntimeError as e:
        print(f"MeCabの実行中にエラーが発生しました: {e}", file=sys.stderr)
        
    return tokens, punct_array

In [22]:
def generate_txt_file_using_mecab(input_txt, path):
    tokens, punct_dict = tokenize_text(input_txt)
    output = ""
    for token in tokens:
        output += token + "\n"
        
    with open(path, "w", encoding="utf-8") as f:
        f.write(output)
    return tokens, punct_dict

In [47]:
import re

m = re.search(r"<unk>|<sil>", r"あ<unk>えお")
print(m.group())

<unk>


In [30]:
from os.path import join, expanduser
import subprocess
import json
import re

SPEAKER_LABELS = ["A", "B"]

def alignment_channel(channel, txt, target_dir_name):
    input_dir_path = join(mfa_input_dir, target_dir_name)
    output_dir_path = join(mfa_output_dir, target_dir_name)
    os.makedirs(input_dir_path, exist_ok=True)
    os.makedirs(output_dir_path, exist_ok=True)
    
    for_align_audio_path = join(input_dir_path, f"{target_dir_name}.wav")
    for_align_txt_path = join(input_dir_path, f"{target_dir_name}.txt")

    sf.write(for_align_audio_path, channel, setting_sr)
    _, punct_array = generate_txt_file_using_mecab(txt, for_align_txt_path)
    subprocess.run([
        "mfa",
        "align",
        input_dir_path,
        "japanese_mfa",
        model_dir,
        output_dir_path,
        "--quiet",
        "--overwrite",
        "--clean",
        "--final_clean",
        "--output_format", "json",
        "--beam", "1000",
        "--retry_beam", "4000",
    ])
    return punct_array

def add_punct(align_json, punct_array):
    segments = align_json["tiers"]["words"]["entries"]
    print("------- before -------")
    print(segments[:20])
    checked_len = 0
    punct_idx = 0
    target_punct = punct_array[punct_idx]
    for idx in range(len(segments)):
        checked_len += segments[idx][2]
        if target_punct[0] == checked_len - 1:
            segments[idx][2] += target_punct[1]
            punct_idx += 1
            target_punct = punct_array[punct_idx]
    print("------- after -------")
    print(segments[:20])
    return align_json

def correct_text(full_text, align_json):
    text_list = [segment[2] for segment in align_json["tiers"]["words"]["entries"]]
    checked_len = 0
    for text in text_list:
        if re.fullmatch(f"^{text}.*", full_text):
            checked_len += len(text)
            full_text = full_text[:checked_len]
        else:
            if re.search(r"<unk>|<sil>", text):
                m = re.search(r"<unk>|<sil>", text)
                matched_text = m.group()
                re.sub(matched_text, "", text)

def parse_ft_json(json_list):
    result = []
    for i in range(len(json_list)):

    segments = json_data[i]["tiers"]["words"]["entries"]
    for segment in segments:
        json.append({
            "speaker": SPEAKER_LABEL[i],
            "word": segment[2],
            "start": segment[0],
            "end": segment[1],
        })
    sorted_result = sorted(result, key=lambda seg: seg["start"])
    return sorted_result

def lst_to_line_str(lst):
    result = ""
    for s in lst:
        result += s
    return result

def alignment_audio_dialogue(text_dialogue_list, audio_path, idx):
    json_list = []
    audio, sr = sf.read(audio_path)
    for i in range(len(SPEAKER_LABELS)):
        channel = audio[:, i]
        
        text_list = []
        for text_dial in text_dialogue_list:
            if text_dial.speaker == SPEAKER_LABELS[i]
                text_list.append(text_dial.text)
            
        speaker_full_text = lst_to_line_str(text_list)
        target_dir_name = f"{SPEAKER_LABELS[i]}{idx}"
        punct_array = alignment_channel(channel, speaker_full_txt, target_dir_name)
        
        json_path = join(mfa_output_dir, target_dir_name, f"{target_dir_name}.json")
        with open(json_path, "r") as f:
            json_data = json.load(f)
        json_data = add_punct(json_data, punct_array)
        json_list.append(json_data)
        
    ft_json = parse_ft_json(json_list)
    return ft_json

## フォルダ初期化

In [24]:
import re

def get_file_name():
    wav_file_pattern = r"^(\d+)\.wav$"
    num = -1
    for file in os.listdir(audio_dir_path):
        if not os.path.exists(os.path.join(audio_dir_path, file)):
            continue
        if not re.match(wav_file_pattern, file):
            continue
    
        match_obj = re.match(wav_file_pattern, file)
        get_number = int(match_obj.groups()[0])
    
        if num < get_number:
            num = get_number
    return num

In [25]:
from glob import glob
import shutil

def delete_files(dir_path):
    shutil.rmtree(dir_path)
    os.makedirs(dir_path)

if IS_REMOVE_EXIST_FILE:
    file_name_num = -1
    for dir_path in base_paths:
        delete_files(dir_path)
else:
    file_name_num = get_file_name()

## メイン処理

In [32]:
%%time

import soundfile as sf
import json

for i in range(file_name_num+1, gen_dial_num+file_name_num+1):

    # テキスト生成
    txt_dialogue_list = gen_txt_dialogue()

    # 音声合成のためのプロンプト生成
    audio_synth_prompt = build_audio_synth_prompt(txt_dialogue_list)

    # 対話テキストを音声合成
    stereo = gen_audio_dialogue(txt_dialogue_list, audio_synth_prompt)
    
    wav_name = f"{i}.wav"
    audio_file_path = os.path.join(audio_dir_path, wav_name)

    # wavファイル出力
    sf.write(audio_file_path, stereo, setting_sr)

    # 音声アラインメント
    json_data = alignment_audio_dialogue(txt_dialogue_list, audio_file_path, i)

    json_name = f"{i}.json"
    json_file_path = os.path.join(json_dir_path, json_name)
    
    # JSON出力
    with open(json_file_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=2)

あなたがこれから音声合成するテキストは以下の対話内容のワンフレーズです。
この対話の文脈に合うように音声合成してください。

<対話内容の全文>
A: 〇〇さん、こんにちは。今日はこれまでのセッションを少し振り返ってみながら、今後のお話もできたらと思うのですが、いかがでしょうか？
B: はい、大丈夫です。もうそんな時期なんですね。あっという間でした。
A: ええ、そうですね。〇〇さんが最初にいらした頃、漠然とした不安や仕事での悩みを抱えていらっしゃいましたね。あれから、ずいぶん色々なことをお話ししてきましたね。
B: はい。まさか、自分がこんなに変われるとは思っていなかったです。以前は本当に毎日が辛くて、どうしたらいいか分からなかったんですけど…。
A: そうなんですね。何か、特に印象に残っていることや、ご自身で「これは変わったな」と感じることはありますか？
B: うーん、そうですね…。やっぱり、自分の感情と行動のつながりを理解できたことが大きいです。以前は、不安になるとすぐ動けなくなってしまっていましたが、最近は「これは不安を感じているんだな」って客観的に見られるようになりました。
A: なるほど、素晴らしい気づきですね。ご自身の感情を客観的に捉えられるようになった、と。それは大きな変化ですね。
B: はい。おかげで、無理に頑張りすぎずに、少し休憩するとか、誰かに相談してみるとか、違う選択肢も考えられるようになりました。以前はそういう発想すらなくて。
A: ええ、以前は一つの選択肢に縛られがちでしたものね。ご自身の変化を、〇〇さんがご自身で感じられているというのは、本当に素晴らしいことです。
B: ありがとうございます。でも、まだ完全に不安がなくなったわけではないので、少し心配な気持ちもあります。
A: そうですね。不安がなくなる、ということではなく、不安とどう向き合っていくか、という視点が大切でしたね。今、〇〇さんには、ご自身でその対処法を見つける力が備わってきていると感じます。これからも、このセッションで得た学びを、ぜひ日常の中で活かしていってくださいね。
B: はい、頑張ります。
A: ええ。今日はここまでにしましょうか。次回のセッションで、また少し今後について具体的に考えていきましょう。何かご不明な点はありませんか？
B: いいえ、大丈夫です。ありがとうご

 INFO     Setting up corpus information...                                      
 INFO     Loading corpus from source files...                                   
 INFO     Found 1 speaker across 1 file, average number of utterances per       
          speaker: 1.0                                                          
 INFO     Initializing multiprocessing jobs...                                  
 WARNING  Number of jobs was specified as 3, but due to only having 1 speakers, 
          MFA will only use 1 jobs. Use the --single_speaker flag if you would  
          like to split utterances across jobs regardless of their speaker.     
 INFO     Normalizing text...                                                   
 INFO     Generating MFCCs...                                                   
 INFO     Calculating CMVN...                                                   
 INFO     Generating final features...                                          
 INFO     Creating corpus sp

[[0.0, 0.03999999910593033, '〇'], [0.03999999910593033, 0.5, 'さん'], [0.5699999928474426, 1.2999999523162842, 'こんにちは'], [1.7300000190734863, 1.9900000095367432, '今日'], [1.9900000095367432, 2.140000104904175, 'は'], [2.140000104904175, 2.4700000286102295, 'これ'], [2.4700000286102295, 2.740000009536743, 'まで'], [2.740000009536743, 2.869999885559082, 'の'], [2.869999885559082, 3.299999952316284, 'セッション'], [3.299999952316284, 3.4000000953674316, 'を'], [3.4000000953674316, 3.809999942779541, '少し'], [3.809999942779541, 4.099999904632568, '<unk>'], [4.400000095367432, 4.519999980926514, 'て'], [4.519999980926514, 4.670000076293945, 'み'], [4.670000076293945, 5.130000114440918, 'ながら'], [5.489999771118164, 5.940000057220459, '今後'], [5.940000057220459, 6.070000171661377, 'の'], [6.070000171661377, 6.110000133514404, 'お'], [6.110000133514404, 6.510000228881836, '話'], [6.510000228881836, 6.639999866485596, 'も'], [6.639999866485596, 6.880000114440918, 'でき'], [6.880000114440918, 7.159999847412109, 'たら'], [7

TypeError: list indices must be integers or slices, not str